In [1]:
import re
import math
import requests
from bs4 import BeautifulSoup
from collections import Counter

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


url = "https://www.gutenberg.org/cache/epub/84/pg84-images.html"
html = requests.get(url, timeout=30).text
soup = BeautifulSoup(html, "html.parser")
raw_text = soup.get_text(separator=" ")

raw_text = re.sub(r"\s+", " ", raw_text).strip()

# 2) Tokenize

tokens = re.findall(r"[a-zA-Z']+|[.!?;,:\-]", raw_text.lower())

PAD = "<PAD>"
UNK = "<UNK>"

min_freq = 5
cnt = Counter(tokens)

vocab = [PAD, UNK] + [w for w, c in cnt.items() if c >= min_freq]
word2id = {w: i for i, w in enumerate(vocab)}
id2word = {i: w for w, i in word2id.items()}

ids = [word2id.get(w, word2id[UNK]) for w in tokens]

window_size = 100
seq_len = window_size - 1

X = []
y = []
for i in range(0, len(ids) - window_size):
    X.append(ids[i:i+seq_len])
    y.append(ids[i+seq_len])

X = torch.tensor(X, dtype=torch.long)
y = torch.tensor(y, dtype=torch.long)

class SeqDS(Dataset):
    def __init__(self, X, y):
        self.X, self.y = X, y
    def __len__(self):
        return self.y.size(0)
    def __getitem__(self, i):
        return self.X[i], self.y[i]

batch_size = 64
loader = DataLoader(SeqDS(X, y), batch_size=batch_size, shuffle=True)

class RNNTextGen(nn.Module):
    def __init__(self, vocab_size, emb_dim=128, hidden_dim=256):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=word2id[PAD])
        self.rnn = nn.RNN(emb_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        e = self.emb(x)
        out, _ = self.rnn(e)
        last = out[:, -1, :]
        logits = self.fc(last)
        return logits

device = "cuda" if torch.cuda.is_available() else "cpu"
model = RNNTextGen(vocab_size=len(vocab)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)

#Train
epochs = 15
for ep in range(1, epochs + 1):
    model.train()
    total_loss = 0.0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * xb.size(0)

    avg_loss = total_loss / len(loader.dataset)
    ppl = math.exp(avg_loss) if avg_loss < 20 else float("inf")
    print(f"Epoch {ep:02d} | loss={avg_loss:.4f} | ppl~{ppl:.1f}")

# 7) Text generation

model.eval()

seed_text = "the creature was"
seed_tokens = re.findall(r"[a-zA-Z']+|[.!?;,:\-]", seed_text.lower())
seed_ids = [word2id.get(w, word2id[UNK]) for w in seed_tokens]

generated = seed_tokens[:]
temperature = 0.9
gen_words = 120

context = seed_ids[:]
if len(context) < seq_len:
    context = [word2id[PAD]] * (seq_len - len(context)) + context
else:
    context = context[-seq_len:]

for _ in range(gen_words):
    x = torch.tensor([context], dtype=torch.long).to(device)
    with torch.no_grad():
        logits = model(x).squeeze(0)
        logits = logits / temperature
        probs = torch.softmax(logits, dim=0)

        next_id = torch.multinomial(probs, num_samples=1).item()

    next_tok = id2word.get(next_id, UNK)
    generated.append(next_tok)

    context = context[1:] + [next_id]

out = []
for t in generated:
    if t in [".", "!", "?", ",", ";", ":"]:
        if out:
            out[-1] = out[-1] + t
        else:
            out.append(t)
    elif t == "-":
        out.append("-")
    else:
        out.append(t)

print("\nGenerated text:\n")
print(" ".join(out))

Epoch 01 | loss=5.0348 | ppl~153.7
Epoch 02 | loss=4.6415 | ppl~103.7
Epoch 03 | loss=4.4769 | ppl~88.0
Epoch 04 | loss=4.4151 | ppl~82.7
Epoch 05 | loss=4.5079 | ppl~90.7
Epoch 06 | loss=4.7056 | ppl~110.6
Epoch 07 | loss=4.7457 | ppl~115.1
Epoch 08 | loss=4.5494 | ppl~94.6
Epoch 09 | loss=4.5064 | ppl~90.6
Epoch 10 | loss=4.6405 | ppl~103.6
Epoch 11 | loss=4.6263 | ppl~102.1
Epoch 12 | loss=4.9803 | ppl~145.5
Epoch 13 | loss=4.6360 | ppl~103.1
Epoch 14 | loss=4.3949 | ppl~81.0
Epoch 15 | loss=4.2356 | ppl~69.1

Generated text:

the creature was dry me, or rather i ought were somewhat, <UNK> were extreme him. i will henry a plain that she was on earth and had been the favourite - <UNK> of the streets little. this frequently were <UNK> the clouds i touched on his feelings on in one, but revenge, dear victor every hate him - fiend of consideration, i ran were, i looked upon their dying continually addressed to hold his body, and day, but there consented by the creation of the habitation